In [129]:
pip install -U spacy

     |████████████████████████████████| 6.5 MB 5.9 MB/s            
     |████████████████████████████████| 917 kB 8.0 MB/s            
     |████████████████████████████████| 146 kB 13.0 MB/s            
     |████████████████████████████████| 11.2 MB 4.4 MB/s            
     |████████████████████████████████| 181 kB 8.8 MB/s            
     |████████████████████████████████| 491 kB 11.4 MB/s            
     |████████████████████████████████| 44 kB 4.8 MB/s             
     |████████████████████████████████| 45 kB 5.1 MB/s             
     |████████████████████████████████| 48 kB 1.1 MB/s             
     |████████████████████████████████| 57 kB 6.9 MB/s             
     |████████████████████████████████| 10.2 MB 185 kB/s             
Note: you may need to restart the kernel to use updated packages.


In [132]:
import torch

# Cek apakah GPU tersedia
print("Is CUDA available:", torch.cuda.is_available())

Is CUDA available: False


In [147]:
import pandas as pd
data = pd.read_csv('clean_data.csv')
train_set = pd.read_csv('train_data_pajak.csv')

In [148]:
data = data.drop('KRITERIA_BARANG', axis=1)

In [149]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import nltk
from nltk import word_tokenize

In [150]:
# X_train, X_test, y_train, y_test = train_test_split(train_set['NAMA_BARANG'], train_set['JENIS'], test_size=0.2, random_state=42)
X_train = train_set['NAMA_BARANG']
X_test = train_set['NAMA_BARANG']
y_train = train_set['JENIS']
y_test = train_set['JENIS']

In [151]:
train_set.head(10)

,NAMA_BARANG,JML_BARANG,JENIS
0,ANALISA SAMPEL RBD PALM STEARIN,1.0,service
1,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,1.0,service
2,Discarging Survey of CPO 2999998 MT at Kabil B...,1.0,service
3,Discarging Survey of CPO 3149193 MT at Kabil B...,1.0,service
4,KALIBRASI SOUNDING TAPE CPO,1.0,service
5,Loading Survey of CPKO 1999887 MT and CPO 2503...,1.0,service
6,CPOCRUDE PALM OIL,93.0,product
7,VITAMIN A PALMITATE 17 mIUg 0418579294 FO HS 2...,250.0,product
8,Lebih Angkut Palm Kernel,378.0,service
9,CPOCrude Palm Oil Kontrak No082XXXXXX20XX,410.0,product


In [ ]:
#vocab = {}
#vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

#X_clean = vectorizer.fit_transform(train_set['NAMA_BARANG'])

#feature_names = vectorizer.get_feature_names()

#for feature, count in zip(feature_names, X_clean.sum(axis=0).A1):
#    vocab[feature] = count
    
#print(vocab)
#word2idx = {w: i for i, w in enumerate(vocab)}

In [107]:
vectorizer = TfidfVectorizer(tokenizer = word_tokenize)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [108]:
nb_classifier = MultinomialNB()

In [109]:
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],  # Adjust the values based on your needs
    'fit_prior': [True, False]
}

In [66]:
data.head()

,NAMA_BARANG,NAMA_BARANG_NO_SPCHAR,JML_BARANG
0,ANALISA SAMPEL RBD PALM STEARIN,ANALISA SAMPEL RBD PALM STEARIN,1.0
1,Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell...,AFLATOXIN B1 QTY 1 SAMPEL PALM KERNEL EXPELL...,1.0
2,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,ANALISA PENGUJIAN INSPEKSI MINYAK KELAPA SAWI...,1.0
3,Discarging Survey of CPO 2999998 MT at Kabil B...,DISCARGING SURVEY OF CPO 2 999 998 MT AT KABIL...,1.0
4,Discarging Survey of CPO 3149193 MT at Kabil B...,DISCARGING SURVEY OF CPO 3 149 193 MT AT KABIL...,1.0


In [110]:
cv_values = [3, 5, 7, 10]

In [111]:
for cv in cv_values:
    grid_search = GridSearchCV(nb_classifier, param_grid, cv=cv, scoring='accuracy')
    grid_result = grid_search.fit(X_train_vect, y_train)

In [112]:
best_alpha = grid_result.best_params_['alpha']

In [113]:
print(f"Best Alpha={best_alpha}, CV={cv}, Best Score: {grid_result.best_score_}")

Best Alpha=0.01, CV=10, Best Score: 0.97


In [114]:
grid_search = GridSearchCV(nb_classifier, param_grid, cv=10, scoring='accuracy')
grid_result = grid_search.fit(X_train_vect, y_train)

In [115]:
y_pred = grid_search.predict(X_test_vect)

In [ ]:
print("\nAccuracy on the test set:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy on the test set: 1.0

Classification Report:
               precision    recall  f1-score   support

     product       1.00      1.00      1.00        70
     service       1.00      1.00      1.00        28

    accuracy                           1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



### Unlabeled Data

In [117]:
test_vect = vectorizer.transform(data['NAMA_BARANG'])
predictions = grid_search.predict(test_vect)

In [118]:
# Add predictions to the original data
data['Predicted_JENIS'] = predictions

In [119]:
data.head()

,NAMA_BARANG,NAMA_BARANG_NO_SPCHAR,JML_BARANG,Predicted_JENIS
0,ANALISA SAMPEL RBD PALM STEARIN,ANALISA SAMPEL RBD PALM STEARIN,1.0,service
1,Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell...,AFLATOXIN B1 QTY 1 SAMPEL PALM KERNEL EXPELL...,1.0,service
2,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,ANALISA PENGUJIAN INSPEKSI MINYAK KELAPA SAWI...,1.0,service
3,Discarging Survey of CPO 2999998 MT at Kabil B...,DISCARGING SURVEY OF CPO 2 999 998 MT AT KABIL...,1.0,service
4,Discarging Survey of CPO 3149193 MT at Kabil B...,DISCARGING SURVEY OF CPO 3 149 193 MT AT KABIL...,1.0,service


In [120]:
data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(data['NAMA_BARANG'], data['Predicted_JENIS'], test_size=0.2, random_state=42)

In [121]:
data_X_train_vect = vectorizer.fit_transform(data_X_train)
data_X_test_vect = vectorizer.transform(data_X_test)

In [122]:
data_grid_search = GridSearchCV(nb_classifier, param_grid, cv=10, scoring='accuracy')
data_grid_result = grid_search.fit(data_X_train_vect, data_y_train)

In [123]:
data_y_pred = grid_search.predict(data_X_test_vect)

In [124]:
print("\nAccuracy on the test set:", accuracy_score(data_y_test, data_y_pred))
print("\nClassification Report:\n", classification_report(data_y_test, data_y_pred))


Accuracy on the test set: 0.990228013029316

Classification Report:
               precision    recall  f1-score   support

     product       0.99      1.00      0.99       232
     service       1.00      0.96      0.98        75

    accuracy                           0.99       307
   macro avg       0.99      0.98      0.99       307
weighted avg       0.99      0.99      0.99       307



In [48]:
#for i in range(len(data)):
  #precision_product = precision_score(data_y_test, data_y_pred, labels=["product"], average=None)
  #print(f"Presisi untuk Data {i}: {precision_product}")

Presisi untuk Data 0: [0.99579832]
Presisi untuk Data 1: [0.99579832]
Presisi untuk Data 2: [0.99579832]
Presisi untuk Data 3: [0.99579832]
Presisi untuk Data 4: [0.99579832]
Presisi untuk Data 5: [0.99579832]
Presisi untuk Data 6: [0.99579832]
Presisi untuk Data 7: [0.99579832]
Presisi untuk Data 8: [0.99579832]
Presisi untuk Data 9: [0.99579832]
Presisi untuk Data 10: [0.99579832]
Presisi untuk Data 11: [0.99579832]
Presisi untuk Data 12: [0.99579832]
Presisi untuk Data 13: [0.99579832]
Presisi untuk Data 14: [0.99579832]
Presisi untuk Data 15: [0.99579832]
Presisi untuk Data 16: [0.99579832]
Presisi untuk Data 17: [0.99579832]
Presisi untuk Data 18: [0.99579832]
Presisi untuk Data 19: [0.99579832]
Presisi untuk Data 20: [0.99579832]
Presisi untuk Data 21: [0.99579832]
Presisi untuk Data 22: [0.99579832]
Presisi untuk Data 23: [0.99579832]
Presisi untuk Data 24: [0.99579832]
Presisi untuk Data 25: [0.99579832]
Presisi untuk Data 26: [0.99579832]
Presisi untuk Data 27: [0.99579832]
Pr

In [22]:
#for i in range(len(data)):
  #precision_service = precision_score(data_y_test, data_y_pred, labels=["service"], average=None)
  #print(f"Presisi untuk Data {i}: {precision_service}")

Presisi untuk Data 0: [0.95652174]
Presisi untuk Data 1: [0.95652174]
Presisi untuk Data 2: [0.95652174]
Presisi untuk Data 3: [0.95652174]
Presisi untuk Data 4: [0.95652174]
Presisi untuk Data 5: [0.95652174]
Presisi untuk Data 6: [0.95652174]
Presisi untuk Data 7: [0.95652174]
Presisi untuk Data 8: [0.95652174]
Presisi untuk Data 9: [0.95652174]
Presisi untuk Data 10: [0.95652174]
Presisi untuk Data 11: [0.95652174]
Presisi untuk Data 12: [0.95652174]
Presisi untuk Data 13: [0.95652174]
Presisi untuk Data 14: [0.95652174]
Presisi untuk Data 15: [0.95652174]
Presisi untuk Data 16: [0.95652174]
Presisi untuk Data 17: [0.95652174]
Presisi untuk Data 18: [0.95652174]
Presisi untuk Data 19: [0.95652174]
Presisi untuk Data 20: [0.95652174]
Presisi untuk Data 21: [0.95652174]
Presisi untuk Data 22: [0.95652174]
Presisi untuk Data 23: [0.95652174]
Presisi untuk Data 24: [0.95652174]
Presisi untuk Data 25: [0.95652174]
Presisi untuk Data 26: [0.95652174]
Presisi untuk Data 27: [0.95652174]
Pr

In [125]:
# Calculate precision for each class
precision_product = precision_score(data_y_test, data_y_pred, labels=["product"], average=None)[0]
precision_service = precision_score(data_y_test, data_y_pred, labels=["service"], average=None)[0]

# Add precision scores to the DataFrame
data.loc[data['Predicted_JENIS'] == 'product', 'Precision_Product'] = precision_product
data.loc[data['Predicted_JENIS'] == 'service', 'Precision_Service'] = precision_service

In [126]:
data['Precision_Product'].fillna('-', inplace=True)
data['Precision_Service'].fillna('-', inplace=True)

In [127]:
data.head()

,NAMA_BARANG,NAMA_BARANG_NO_SPCHAR,JML_BARANG,Predicted_JENIS,Precision_Product,Precision_Service
0,ANALISA SAMPEL RBD PALM STEARIN,ANALISA SAMPEL RBD PALM STEARIN,1.0,service,-,1
1,Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell...,AFLATOXIN B1 QTY 1 SAMPEL PALM KERNEL EXPELL...,1.0,service,-,1
2,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,ANALISA PENGUJIAN INSPEKSI MINYAK KELAPA SAWI...,1.0,service,-,1
3,Discarging Survey of CPO 2999998 MT at Kabil B...,DISCARGING SURVEY OF CPO 2 999 998 MT AT KABIL...,1.0,service,-,1
4,Discarging Survey of CPO 3149193 MT at Kabil B...,DISCARGING SURVEY OF CPO 3 149 193 MT AT KABIL...,1.0,service,-,1


In [128]:
 data.to_csv('2_output_results.csv', index=False)

In [135]:
#precision_product = precision_score(data_y_test, data_y_pred, labels=["product"], average=None)
#precision_service = precision_score(data_y_test, data_y_pred, labels=["service"], average=None)

# Print precision for each class
#for i, (prod_precision, serv_precision) in enumerate(zip(precision_product, precision_service)):
 #   print(f"Presisi untuk NAMA_BARANG ke-{i + 1} (Product): {prod_precision}")
  #  print(f"Presisi untuk NAMA_BARANG ke-{i + 1} (Service): {serv_precision}")

In [136]:
#for i in range(len(data)):
 #   prediction = data['Predicted_JENIS'][i]  # Prediksi dari model
  #  precision = precision_score([prediction], [prediction], labels=["product", "service"], average=None)
   # print(f"Presisi untuk Data {i}: {precision}")

In [137]:
#precision_per_data = precision_score(data_y_test, data_y_pred, average=None)
#print(precision_per_data)

In [138]:
#for i in data['Predicted_JENIS']:
 # if (data['Predicted_JENIS'][i]=="product"):
  #  precision_product = precision_score(data_y_test, data_y_pred, labels=["product"], average=None)
   # print(f"Presisi untuk Data {i}: {precision_service}")
  #elif (data['Predicted_JENIS'][i]=="service"):
   # precision_service = precision_score(data_y_test, data_y_pred, labels=["service"], average=None)
#    print(f"Presisi untuk Data {i}: {precision_service}")